In [1]:
# production code
from pydantic import BaseModel, validator

from typing import List


# base classes

class ValueObject(BaseModel):
    """
    Value objects:

    - Are not unique.
    - Are immutable. Their data (class attributes) may not change over time.
    - May only contain business logic w.r.t. data (class attribute) validation (defensive programming).

    ATTENTION:
    As long as the frozen config option is in beta (https://docs.pydantic.dev/usage/model_config/#options)
    it's safer to use pydantic.dataclasses.dataclass (drop-in replacement for dataclasses with validation and de-/serialization)
    instead. Support for inheritance with dataclass is very limited however and you cannot inherit from a parent dataclass then.

    from pydantic.dataclasses import dataclass

    @dataclass(frozen=True)
    class OrderLine:
        ...

    """
    ...

    class Config:
        frozen = True


class Entity(BaseModel):
    """Entities:

    - Are unique. Represent domain objects which are unique over time.
    - Are mutable. Their data (class attributes) may change over time.
    - May only contain business logic w.r.t. data (class attribute) validation (defensive programming).
    """
    ...


class AggregateRoot(Entity):
    """AggregateRoot:

    - Is an entity which acts as aggregate root.
    - Are mutable. Their data (class attributes) may change over time.
    - Must provide methods which allow to change it's data.
    - Must contain business logic which enforces data integrity.
    """
    ...


# entities

class Order(AggregateRoot):
    """
    Data integrity requirements:
    - Ensures that order lines of the same products are summarized (instead of new order lines with same product_ids added).
    - Ensures that overall_price is consistent with the sum of product prices of all order lines.
    """
    order_lines: List['OrderLine'] = []  # We cannot use set() here cause OrderLine contains mutable products.
    overall_price: int = 0

    def add_order_line(self, order_line: 'OrderLine') -> None:
        for index, existing_order_line in enumerate(self.order_lines):
            if order_line.product.product_number == existing_order_line.product.product_number:
                # Increase product count of existing order line instead of adding a new order line with the same product.
                # Cause order lines are immutable value objects we need to create a new one and remove the old one.
                self.order_lines.append(
                    OrderLine(
                        product=Product(product_number=existing_order_line.product.product_number, price=existing_order_line.product.price),
                        product_count=existing_order_line.product_count + order_line.product_count,
                    )
                )
                self.order_lines.remove(existing_order_line)
                # Ensure data integrity of overall_price.
                self.overall_price += order_line.product_count * order_line.product.price
                return
        self.order_lines.append(order_line)
        # Ensure data integrity of overall_price.
        self.overall_price += order_line.product_count * order_line.product.price

    def remove_order_line(self, product_number: int) -> None:
        for index, existing_order_line in enumerate(self.order_lines):
            if product_number == existing_order_line.product.product_number:
                self.order_lines.remove(existing_order_line)
                # Ensure data integrity of overall_price.
                self.overall_price -= existing_order_line.product_count * existing_order_line.product.price
                return
        return


class Product(Entity):
    """
    Uniqueness:
    - Is unique cause of product_number uniquely identifying a product.

    Mutability:
    - Price of product can change over time.
    """
    product_number: int
    price: int

    @validator('product_number')
    def validate_product_number(cls, value):
        product_number_min = 0
        product_number_max = 999999
        if value < product_number_min or value > product_number_max:
            raise ValueError(f'product_number is {value} but must be between {product_number_min} and {product_number_max}.')
        return value

    @validator('price')
    def validate_price(cls, value):
        price_min = 1
        if value < price_min:
            raise ValueError(f'price is {value} but must be at least {price_min}.')
        return value

# value objects

class OrderLine(ValueObject):
    """
    Note:
        Value object may not change over time itself (mutability: immutable) but data it contains (product)
        refers to a data type whose data may change over time (product.price).
    """
    product: 'Product'
    product_count: int

    @validator('product_count', always=True)
    def validate_product_count(cls, value):
        if value <= 0:
            raise ValueError(f'product_count is {value} but must be greater than 0.')
        return value

It's should not be possible to create invalid entities (`Product`).

In [2]:
Product(product_number=123456)

ValidationError: 1 validation error for Product
price
  field required (type=value_error.missing)

In [ ]:
Product(product_number=123456, price=-1)

In [ ]:
Product(product_number=1000000, price=1)

Valid entities (`Product`) ...

In [ ]:
first_product = Product(product_number=123456, price=1)
first_product

... may be changed after creation.

In [ ]:
first_product.price = 2
first_product

It should not be possible to create invalid value objects (`OrderLine`) as well.

In [ ]:
OrderLine(product=Product(product_number=123456, price=100), product_count=0)

Valid value objects (`OrderLine`) ...

In [ ]:
immutable_order_line = OrderLine(product=Product(product_number=123456, price=100), product_count=1)
immutable_order_line

... may not be changed after creation.

In [ ]:
immutable_order_line.product_count = 2

The entity acting as aggregate root...

In [ ]:
order = Order()
order

...allows to add data and keeps overall_price in sync with single order lines (`1*100 + 2*200 -> 500`)...

In [ ]:
order.add_order_line(OrderLine(product=Product(product_number=123456, price=100), product_count=1))
order.add_order_line(OrderLine(product=Product(product_number=654321, price=200), product_count=2))
order

...keeps order lines consistent w.r.t. product numbers (product count of already existing order line is increased instead of new order line with same product number added) and overall price is still in sync (`100*1 + 200*3 -> 700`).

In [ ]:
order.add_order_line(OrderLine(product=Product(product_number=654321, price=200), product_count=1))
order

We can remove orders as well and overall price is still in sync.

In [ ]:
order.remove_order_line(123456)
order